# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
api_url

'https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448'

In [5]:
data = requests.get(api_url)
data.status_code == 200

True

In [6]:
data = data.json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
price = data['latestPrice']
market_cap = data['marketCap']
market_cap

2908354345182

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
columns = ['Ticker', 'Stock Price', 'Market Capitialization', 'Number of shares to buy']
final_df = pd.DataFrame(columns=columns)
final_df

,Ticker,Stock Price,Market Capitialization,Number of shares to buy


In [9]:
final_df.append(pd.Series([symbol, price, market_cap, 'N/A'], index=columns ), ignore_index=True)

,Ticker,Stock Price,Market Capitialization,Number of shares to buy
0,AAPL,178.68,2908354345182,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [10]:
final_df

,Ticker,Stock Price,Market Capitialization,Number of shares to buy


In [11]:
for stock in stocks['Ticker'][:5]:
    symbol = 'AAPL'
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_df = final_df.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ], index= columns), ignore_index= True)

In [12]:
final_df

,Ticker,Stock Price,Market Capitialization,Number of shares to buy
0,A,162.82,48167848255,N/A
1,AAL,17.15,11473702043,N/A
2,AAP,237.02,15079966824,N/A
3,AAPL,182.19,2951540154327,N/A
4,ABBV,131.17,225940213446,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [13]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [14]:
symbols_group = list(chunks(stocks['Ticker'], 100))
final_dataframe = pd.DataFrame(columns=columns)
for i in range(len(symbols_group)):
    symbols_string = ','.join(symbols_group[i])
    batch_api_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call).json()
    for symbol in symbols_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ], index=columns), ignore_index=True)
final_dataframe

,Ticker,Stock Price,Market Capitialization,Number of shares to buy
0,A,160.57,48305951081,N/A
1,AAL,17.49,11485975472,N/A
2,AAP,236.65,14842261087,N/A
3,AAPL,176.30,3003929700229,N/A
4,ABBV,127.71,230210814823,N/A
...,...,...,...,...
500,YUM,132.74,39659776604,N/A
501,ZBH,126.80,26629699537,N/A
502,ZBRA,633.22,32780475110,N/A
503,ZION,61.58,9957048082,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [15]:
portfolio_size = input("Enter the size of your portfolio: ")
try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease enter an integer")
    portfolio_size = input("Enter the size of your portfolio: ")
    val = float(portfolio_size)

Enter the size of your portfolio: 10000000


In [16]:
position_size = val / len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of shares to buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitialization,Number of shares to buy
0,A,160.57,48305951081,123
1,AAL,17.49,11485975472,1132
2,AAP,236.65,14842261087,83
3,AAPL,176.30,3003929700229,112
4,ABBV,127.71,230210814823,155
...,...,...,...,...
500,YUM,132.74,39659776604,149
501,ZBH,126.80,26629699537,156
502,ZBRA,633.22,32780475110,31
503,ZION,61.58,9957048082,321


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [17]:
writer = pd.ExcelWriter('recomended trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recomended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [18]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({
    'font_color' : font_color,
    'bg_color': background_color,
    'border': 1
})
dollar_format = writer.book.add_format({
    'num_format' : '$0.00',
    'font_color' : font_color,
    'bg_color': background_color,
    'border': 1
})
integer_format = writer.book.add_format({
    'num_format' : '0',
    'font_color' : font_color,
    'bg_color': background_color,
    'border': 1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [19]:
# writer.sheets['Recomended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recomended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recomended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recomended Trades'].set_column('D:D', 18, string_format)
# writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [20]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitialization', dollar_format],
    'D':['Number of shares to buy', integer_format],
}

for column in column_formats.keys():
    writer.sheets['Recomended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recomended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [21]:
writer.save()